In [1]:
import os
import sys
import glob

import scipy.ndimage as ndimage
import scipy.spatial as spatial
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings( "ignore")
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload

import tifffile as tf
import citrus_utils as vitaminC

In [2]:
reload(vitaminC)

<module 'citrus_utils' from '/home/ejam/documents/citrus/citrus/citrus_utils.py'>

In [3]:
comps_src = '../preproc/comps/'
tissue_src = '../preproc/tissue/'
oil_src = '../preproc/oil/'
bname = 'SW03_CRC1241-B_12B-4-3'

In [4]:
src = comps_src + bname
comp_files = sorted(glob.glob(src + '/*.tif'))
print(comp_files)

i = 0 
fname = os.path.split(comp_files[i])[1]
lname = os.path.splitext(fname)[0].split('_')[-1]
print('\n',lname)
#for lname in os.listdir(src):

tissue_filename = tissue_src + bname + lname + '/' + bname + '_' + lname + '_'
print(tissue_filename)

['../preproc/comps/SW03_CRC1241-B_12B-4-3/SW03_CRC1241-B_12B-4-3_L00.tif', '../preproc/comps/SW03_CRC1241-B_12B-4-3/SW03_CRC1241-B_12B-4-3_L01.tif', '../preproc/comps/SW03_CRC1241-B_12B-4-3/SW03_CRC1241-B_12B-4-3_L02.tif']

 L00
../preproc/tissue/SW03_CRC1241-B_12B-4-3L00/SW03_CRC1241-B_12B-4-3_L00_


In [5]:
tissue_size = []
tissue_size.append(bname)
tissue_size.append(lname)
tissue = tf.imread(comps_src + bname + '/' + bname + '_' + lname + '.tif')
tissue_size.append(np.sum(tissue > 10))

In [6]:
tissue_types = ['exocarp', 'endocarp', 'rind']

for tss in tissue_types:
    tissue = tf.imread(tissue_filename + tss + '.tif')
    tissue_size.append(np.sum(tissue > 10))

In [7]:
tissue_size

['M04_CRC3752_12B-28-7', 'L00', 142152844, 22243784, 99170635, 30940687]

In [8]:
spine = tf.imread(tissue_filename + 'spine.tif')
tissue_size.append(np.sum(spine > 10))

In [9]:
spine[spine > 0] = 1
meso = tissue*(1 - spine)
tendo = vitaminC.get_largest_element(meso)
meso *= tendo

5400 components
[24566811     2622     2573     2381     2122     2106     1845     1647
     1526     1511     1489     1465     1372     1177     1168     1158
     1053      919      840      808]


In [10]:
tissue_size.append(np.sum(meso > 10))

In [11]:
citrus_oil = oil_src + bname + '/' + lname + '/' + bname + '_'
tissue = tf.imread(citrus_oil + 'glands.tif')
tissue_size.append(np.sum(tissue > 10))

In [12]:
tissue_size.append(vitaminC.buf_count_newlines_gen(citrus_oil + 'glandscenter.csv'))

In [13]:
dst = '../preproc/allometry/data/'
df = pd.DataFrame(tissue_size).T

filename = dst + bname + '_' + lname + '.csv'
df.to_csv(filename, index = False, header=False)